# Libraries & Packages

Prerequites:
```shell
sudo pip install -r ./requirements.txt
```

In [2]:
import pandas

# File Loading & initial overview

In [20]:
def df_initial_inspection(df: pandas.DataFrame) -> str:
    output = ""

    first_rows = df.head(5)
    column_cardinality = df.nunique()
    total_shape = df.shape
    duplicates = df[df.duplicated(keep=False)]
    unique_shape = df.drop_duplicates().shape

    return f"""First 5 rows:
{first_rows}

Column Cardinality:
{column_cardinality}

Size (with and without duplicates):
{total_shape}
{unique_shape}

Duplicate records:
{duplicates}"""

## Locations

In [21]:
locations = pandas.read_csv("../data/locations.csv")

print(df_initial_inspection(locations))

First 5 rows:
   zip_code        location
0     10506  King's Landing
1     80976           Dorne
2     78956         Braavos
3     67305       Harrenhal
4     25089      Winterfell

Column Cardinality:
zip_code    8
location    8
dtype: int64

Size (with and without duplicates):
(8, 2)
(8, 2)

Duplicate records:
Empty DataFrame
Columns: [zip_code, location]
Index: []


**Notes:**
* No duplicates, 1 to 1 match between a zip code (integer) and an area (string)
* Will most likely be used as input for a geographical dimension table in our data model

## Companies

In [27]:
companies = pandas.read_json("../data/companies.json")

companies["Establishment Date"] = pandas.to_datetime(companies["Establishment Date"], unit="ms")

print(df_initial_inspection(companies))

First 5 rows:
   Company ID               Company Name Establishment Date  \
0           0                 Delta Inc.         2019-09-13   
1           1                 Best Corp.         2022-02-28   
2           2                 Delta Inc.         2019-09-07   
3           3            Fast Track Inc.         2021-03-10   
4           4  Bright Future Enterprises         2019-01-02   

   Number of Employees  
0                  460  
1                  302  
2                  353  
3                   92  
4                  246  

Column Cardinality:
Company ID             10
Company Name            6
Establishment Date     10
Number of Employees    10
dtype: int64

Size (with and without duplicates):
(10, 4)
(10, 4)

Duplicate records:
Empty DataFrame
Columns: [Company ID, Company Name, Establishment Date, Number of Employees]
Index: []


In [24]:
companies[companies["Company Name"] == "Delta Inc."].sort_values(by=["Establishment Date"])

,Company ID,Company Name,Establishment Date,Number of Employees
6,6,Delta Inc.,2018-07-09,434
2,2,Delta Inc.,2019-09-07,353
0,0,Delta Inc.,2019-09-13,460
9,9,Delta Inc.,2022-10-30,440


**Notes:**
* We have here some properties of an `Company` entity: a unique ID, a name (which contains duplicates) and establishment date.
  * `Delta Inc.` is a clear outlier in this data set as there are 4 records of it. 
* We also have one summary statistics about each company: its number of employees.

**Open questions:**
1. The fact that Delta Inc. shows up 4 times raises some questions. Potential explanations could be:
* This is not a one-row-per-entity table, but rather tracking changes over time? 
* Delta Inc. has multiple branches in different territories that opened at different times and operate independently, hence individual properties for each. In that case the data misses additional information to make the distinction for analysis.
* Delta Inc. shut down and re-opened multiple times, which seems unlikely. A `Closure date` timestamp or even an `Is operating` boolean would clarify this aspect.

The last file might help clarify the above questions. Usually, this is where business knowledge from stakeholders and/or documentation come in handy to help clarify such questions.

## Jobs

In [26]:
jobs = pandas.read_json("../data/jobs.json")

print(df_initial_inspection(jobs))
jobs["state"].unique()

First 5 rows:
            posted_at  id      state    zip   price
0 2022-04-13 11:43:00   1    expired  10506  119.34
1 2021-01-20 04:54:00   2  cancelled  10506  197.89
2 2021-07-21 15:29:00   3    expired  35786  335.85
3 2021-06-13 20:11:00   4    expired  78956  150.00
4 2022-06-23 06:52:00   5    expired  78956  158.09

Column Cardinality:
posted_at    100
id           100
state          5
zip            5
price         98
dtype: int64

Size (with and without duplicates):
(102, 5)
(100, 5)

Duplicate records:
              posted_at  id      state    zip   price
2   2021-07-21 15:29:00   3    expired  35786  335.85
96  2022-12-29 16:05:00  97  cancelled  78956   11.83
100 2022-12-29 16:05:00  97  cancelled  78956   11.83
101 2021-07-21 15:29:00   3    expired  35786  335.85


array(['expired', 'cancelled', 'posted', 'osted', 'canceled'],
      dtype=object)

**Notes:**
* This file seems to keep records of the `Job` (job postings) entity with some of its properties:
  * Id
  * Price
  * Zip (will allow connection to `locations`)
  * Posted at date and time
  * Status (expired, open or cancelled, with a few additional values due to typos and american vs british english spellings)
* Duplicates were identified in the data with ID 3 and 97 (respectively rows {2, 101} and {96, 101})
* A few records have no prices attached to them. Not necessarily a data quality issue: it could be a non-paid job for example?
* There appears to be no clear obvious connection between this data and `companies`.

**Open questions:**
* Based on the rest of the questions in the exercise, it appears necessary to have a connection between `jobs` and `companies` but there doesn't to be an immediate connection between the 2.
* I will make the following assumption in order to be able to move forward with the rest of the work: only one company can have jobs published at a single time. That way based on the history in `companies`, I'll be able to derive the time ranges in which companies had job postings. Combining this with the `posted_at` from `jobs`, I'll be able to map jobs to companies.